In [2]:
import os
import numpy as np
import pandas as pd

import scipy.stats as st

**По данным опроса, 75% работников ресторанов утверждают, что испытывают на работе существенный стресс, оказывающий негативное влияние на их личную жизнь. Крупная ресторанная сеть опрашивает 100 своих работников, чтобы выяснить, отличается ли уровень стресса работников в их ресторанах от среднего. 67 из 100 работников отметили высокий уровень стресса.**

**Посчитайте достигаемый уровень значимости, округлите ответ до четырёх знаков после десятичной точки.**

In [4]:
round(st.binom_test(67, 100, .75), 4)

0.0822

**The Wage Tract — заповедник в округе Тома, Джорджия, США, деревья в котором не затронуты деятельностью человека со времён первых поселенцев. Для участка заповедника размером 200х200 м имеется информация о координатах сосен (sn — координата в направлении север-юг, we — в направлении запад-восток, обе от 0 до 200).**

**Проверим, можно ли пространственное распределение сосен считать равномерным, или они растут кластерами.**

**Загрузите данные, поделите участок на 5х5 одинаковых квадратов размера 40x40 м, посчитайте количество сосен в каждом квадрате (чтобы получить такой же результат, как у нас, используйте функцию scipy.stats.binned_statistic_2d).**

**Если сосны действительно растут равномерно, какое среднее ожидаемое количество сосен в каждом квадрате? В правильном ответе два знака после десятичной точки.**

In [51]:
with open(os.path.join('data', 'pines.txt'), 'r') as txt:
    data = np.array([row.split('\t') for row in txt.read().split('\n') if len(row.split('\t')) == 2])[1:, :].astype(float)

In [52]:
data, data.shape

(array([[200. ,   8.8],
        [199.3,  10. ],
        [193.6,  22.4],
        ...,
        [156.7, 155.3],
        [154.1, 150.8],
        [ 87.7, 200. ]]), (584, 2))

In [108]:
xbins = np.arange(0, 240, 40)
ybins = np.arange(0, 240, 40)
res = st.binned_statistic_2d(x=data[:, 0], y=data[:, 1], values=range(201), statistic='count', bins=(xbins, ybins))

In [109]:
res.statistic

array([[18., 26., 29., 25., 20.],
       [14., 12., 22., 34., 25.],
       [12., 24., 15., 50., 37.],
       [ 8., 19., 31., 50.,  7.],
       [ 7.,  8., 37., 27., 27.]])

In [110]:
res.statistic.mean()

23.36

**Чтобы сравнить распределение сосен с равномерным, посчитайте значение статистики хи-квадрат для полученных 5х5 квадратов. Округлите ответ до двух знаков после десятичной точки.**

In [121]:
observed = res.statistic
expected = np.random.uniform(high=observed.max(), size=(5, 5))
observed, expected

(array([[18., 26., 29., 25., 20.],
        [14., 12., 22., 34., 25.],
        [12., 24., 15., 50., 37.],
        [ 8., 19., 31., 50.,  7.],
        [ 7.,  8., 37., 27., 27.]]),
 array([[16.57738582, 18.50251329, 40.05543587, 47.88482947, 39.77488371],
        [44.88884572, 22.18076861, 11.19651579, 31.07181466,  7.56356095],
        [33.17571701, 36.4447366 , 14.9343747 , 14.78806552, 24.28444426],
        [ 8.16880204, 23.05904079, 47.44186416, 39.81643045, 47.45713564],
        [25.40163624, 13.44483487, 14.04371695,  1.94829048, 31.28561442]]))

In [131]:
expected = [observed.mean() for _ in range(25)]
chi2_result = st.chisquare(observed.flatten(), np.array(expected), ddof=1, axis=None)
round(chi2_result.statistic, 2)

150.59

**Насколько велико это значение? Если нулевая гипотеза справедлива, с какой вероятностью его можно было получить случайно?**

**Нулевое распределение статистики — хи-квадрат с 25-1=2425−1=24 степенями свободы (поскольку у равномерного распределения, с которым мы сравниваем данные, нет ни одного оцениваемого по выборке параметра, число степеней свободы K-1K−1, где KK — количество интервалов).**

**Посчитайте достигаемый уровень значимости.**

**Если вы используете функцию scipy.stats.chi2.cdf, в качестве значения параметра df нужно взять 24 (это число степеней свободы); если функцию scipy.stats.chisquare — параметр ddof нужно брать равным 0 (это как раз количество параметров теоретического распределения, оцениваемых по выборке).**

**Отвергается ли гипотеза равномерности на уровне значимости 0.05?**

In [133]:
chi2_result.pvalue

9.87806178820889e-21

Достигаемый уровень значимости меньше 0.05, гипотеза равномерности отвергается.